# Queueing theory - M/M/1 example

#### Issues
* The arrivals followed a poisson distribution, but in this setup of the package we enter the interarrival times which follow an exponential one. We may need to look at how generic this approach will be for other types of distributions. How will that work for Erlang distributions.
* For 1 M customers the script allready becomes reasonably slow. Perhaps we should make logging optional and have a couple of key running means. Interesting to see if that will speed things up.
* Maybe we could then implement iteration until e.g. the mean of the distribution is accurately represented to lets say 0.001 or 0.0001.
* Maybe we should not allow entering of distributions, but rather steer the simulations with the Kendall codes. Just provide M/M/1 and the nr of customers should be enough for example.
* We want to have a notebook with D/D/1 but then also include quay length (to match te example of the P&W II case). This needs some additional thinking. Maybe this can be solved with length of the queue where the quay is a qeueu lenght in m and a vessel can takes several meters of queue.
* Using multiple servers seems to work, but that still needs to be thoroughly checked
* Need to make sure we can extract the same statistics from the simulation as from the analytical result to enable comparison (validation check)

### 1. Imports

In [4]:
from scipy import stats

import pandas as pd
import numpy as np

import openqtsim

import matplotlib.pyplot as plt

### 2. Prepare inputs

#### Specify the long term average IAT and ST in seconds

In [5]:
# arrivals and services
arrivals_per_hour = 8  # lambda
services_per_hour = 9  # mu

# inter arrival times and service times
aver_IAT_in_secs = 3600 / arrivals_per_hour
aver_ST_in_secs = 3600 / services_per_hour

#### Define the queue object input

In [6]:
A = openqtsim.arrival_process("M", stats.expon(scale = aver_IAT_in_secs))
S = openqtsim.service_process("M", stats.expon(scale = aver_ST_in_secs))
c = 1

TypeError: 'module' object is not callable

#### Create the queue object and use it to create a simulating object

In [ ]:
# todo: enable deterministic analyses by adding a list of 'customer_name#', 'interarrival#' and 'servicetime#'
q = openqtsim.queue(A, S, c)
sim = openqtsim.simulation(q)

### 3. Run the simulation

In [ ]:
%%time
print('Start simulation of "{}" queue'.format(q.kendall_notation))
sim.run(1000) # nr is the number of customers that will arrive at the server
sim.return_log(10)

### 4. Analyse and visualise results

#### Get statistics

In [ ]:
print('--- input statistics --------')
print("aver_IAT_in_secs: {:.2f} [seconds]".format(aver_IAT_in_secs))
print("aver_ST_in_secs: {:.2f} [seconds]".format(aver_ST_in_secs))
print('')

print('--- simulation statistics ---')
sim.get_stats()

**Analytical solution: M_M_1_inf_inf_FIFO**

    arrival rate: 8.0000 per hour
    service rate: 9.0000 per hour
    rho: 0.8889

    P_0: probability that there are 0 customers in the system = 0.1111
    P_1: probability of 1 customers in the system = 0.0988
    P_10: probability of 10 customers in the system = 0.0342

    no_q: probability that there is nobody in the queue (P_0+P_1) = 0.2099
    util_sys: system utilisation rate  (P_(n>=1)) = 0.8889
    util_ser: server utilisation rate (P_(n>=2)) = 0.7901

    L_s: the long term average number of customers present in the system = 8.0000
    L_q: the long term average number of customers present in the queue = 7.1111
    W_s: the long term average waiting time in the system = 1.0000 hour (= 60.0 minutes)
    W_q: the long term average waiting time in the queue = 0.8889 hour (= 53.3 minutes)

#### Return log

In [ ]:
sim.return_log(15)
# c = customer
# IAT = inter arrival time
# ST = service time
# AT = arrival time
# TSB = time service begins
# TSE = time service ends
# TCSS = time customer spends in the system
# TCWQ = time customer waits in the queue
# ITS = idle time of the server
# QL = queue length

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))

fig.suptitle('Overview of stochastic variables')
ax1.hist(sim.log["IAT"],1000); ax1.set_title('IAT');
ax2.hist(sim.log["ST"],1000); ax2.set_title('ST');

## Sandbox